# Project 10: Liquidation Visualizer
## Analyzing Liquidation Patterns Across Different Leverage Levels

This notebook explores liquidation data from crypto exchanges to understand when traders get "rekt".

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

from src.data_fetcher import LiquidationDataFetcher
from src.data_processor import LiquidationDataProcessor
from src.visualizer import LiquidationVisualizer
import config

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Imports successful!")

## 2. Fetch Liquidation Data

In [ ]:
# Initialize data fetcher
fetcher = LiquidationDataFetcher(api_key=config.API_KEY)

# Fetch data
raw_data = fetcher.fetch_liquidation_map(
    exchange="Bi**ce",
    pair="BTC/USDT",
    time_type="1D"
)

print("✅ Data fetched successfully!")
print(f"Timestamp: {raw_data['timestamp']}")

## 3. Process Data

In [ ]:
# Initialize processor
processor = LiquidationDataProcessor(raw_data)

print(f"Current BTC Price: ${processor.current_price:,.2f}")
print(f"\nProcessed leverage levels: {list(processor.leverage_data.keys())}")

## 4. Explore Data

In [ ]:
# View 100x leverage data
df_100x = processor.get_leverage_data("100x")
print("Sample of 100x Leverage Data:")
df_100x.head(10)

In [ ]:
# Summary statistics
summary = processor.get_liquidation_summary()
print("\nLiquidation Summary - All Leverage Levels:")
summary

## 5. Visualizations

In [ ]:
# Initialize visualizer
visualizer = LiquidationVisualizer(processor)

### 5.1 Liquidation Heatmap - All Leverage Levels

In [ ]:
visualizer.plot_liquidation_heatmap()

### 5.2 Compare Leverage Levels

In [ ]:
visualizer.compare_leverage_levels()

### 5.3 Critical Liquidation Zones

In [ ]:
visualizer.identify_liquidation_zones(leverage="100x", top_n=10)

### 5.4 Interactive Heatmap

In [ ]:
visualizer.create_interactive_heatmap(leverage="100x")

## 6. Analysis & Insights

In [ ]:
# Analyze each leverage level
for leverage in config.LEVERAGE_LEVELS:
    stats = processor.calculate_statistics(leverage)
    print(f"\n{'='*60}")
    print(f"{leverage} Leverage Statistics")
    print(f"{'='*60}")
    print(f"Total Liquidation Amount: ${stats['total_liquidation_amount']:,.0f}")
    print(f"Long Liquidations: ${stats['long_liquidation_amount']:,.0f}")
    print(f"Short Liquidations: ${stats['short_liquidation_amount']:,.0f}")
    print(f"Long/Short Ratio: {stats['long_short_ratio']:.2f}")
    print(f"Number of Levels: {stats['num_liquidation_levels']}")
    print(f"Price Range: ${stats['price_range'][0]:,.0f} - ${stats['price_range'][1]:,.0f}")

## 7. Key Findings

### Observations:
1. **Higher leverage = tighter liquidation zones**: 100x leverage shows liquidations clustered very close to current price
2. **Long vs Short imbalance**: Analyze which side has more liquidation risk
3. **Critical price levels**: Identify key support/resistance based on liquidation clusters

### Trading Implications:
- Large liquidation zones can act as magnets for price movement
- Cascading liquidations can cause rapid price swings
- Understanding liquidation levels helps with risk management